# Como crear: Un flujo de trabajo computacional reproducible usando Jupyter notebook
Content under Creative Commons Attribution license CC-BY-NC-SA 4.0   
Code under GNU-GPL v3 License  
© 2019 Serena Bonaretti
---

Este cuaderno es un ejemplo de cómo crear un flujo de trabajo reproducible utilizando el cuaderno Jupyter y python.  
Hay 4 pasos que hacen que este flujo de trabajo sea reproducible:  
1. 1. [Descargar automáticamente los datos de un repositorio](#download)  
2. 2. [Automatizar la manipulación de datos](#manipulation)
3. [Definir semillas para generar números aleatorios](#seeds)
4. 4. [Imprimir dependencias](#dependencies) 


El ejemplo codificado en este cuaderno consiste en la creación de un mapa interactivo de revisión bibliográfica sobre la segmentación del cartílago femoral de la rodilla. El mapa muestra el tipo de algoritmo, la localización, el primer autor y el año de publicación.
Los datos de entrada son un archivo `.csv` en Zenodo (www.doi.org/10.5281/zenodo.3553483) que contiene datos sobre la literatura de la segmentación del cartílago femoral de la rodilla.

---

## Instalar librearías necesarias

In [18]:
#pip install wget altair vega_datasets vega watermark

## Importamos librerías

In [19]:
import wget # to download from zenodo
import pandas as pd  
import numpy  as np
# from   geopy.geocoders import Nominatim
import altair as alt
import vega 
from   vega_datasets import data # for state contours 

In [20]:
#pip install vega
# alt.renderers.enable('notebook'); # for rendering in jupyter notebook

---
<a name = "download"></a>
## 1. Descargando datos automáticamente desde un repositorio


Los datos de entrada deben estar en un repositorio que proporcione un **identificador digital de objeto (DOI)** persistente para que los datos estén disponibles en el futuro. Se *desaconseja* compartir datos de **repositorios personales** porque los enlaces tienden a borrarse, comprometiendo así la reproducibilidad del flujo de trabajo.
- Descargue el archivo `cart_segm_literature.csv` de Zenodo, abra el archivo descargado y muestre el contenido


In [48]:
# Nombre de archivo y URL de Zenodo
file_name  = "cart_segm_literature.csv" # Nombre como se llama el archivo
zenodo_url = "https://zenodo.org/record/3553483/files/" # Esta es la url de la página de Zenodo donde se encuentra el archivo, debes adicionarle files/ al final

In [49]:
# download
wget.download(zenodo_url + file_name, "./" + file_name) # entrada y salida

'./cart_segm_literature (1).csv'

- Carga el archivo `cart_segm_literature.csv` 
- Muestra los datos en el archivo

In [50]:
# cargar el archivo
literature = pd.read_csv("./" + file_name)

In [51]:
literature

,algorithm_type,bibtex_id,author_1,author_2,author_3,author_4,author_5,author6,author_7,author_8,affiliation_last_author,city_last_author,country_last_author,title,journal,year,link_to_pubmed,latitude,longitude
0,active contours,Solloway_1997,﻿Solloway S,Hutchinson CE,Waterton JC,Taylor CJ,NaN,NaN,NaN,NaN,"Department of Medical Biophysics, University o...",Manchester,UK,﻿The use of active shape models for making thi...,﻿Magn Reson Med 37(6):943–952,1997,https://www.ncbi.nlm.nih.gov/pubmed/9178247,53.479489,-2.245115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,hybrid,Dam_2015,Dam ﻿E B,Lillholm M,Marques J,Nielsen M,NaN,NaN,NaN,NaN,Biomediq and University of Copenhagen,Copenhagen,Denmark,Automatic segmentation of high- and low-field ...,"﻿SPIE Journal of Medical Imaging, vol. 2, pp....",2015,https://www.ncbi.nlm.nih.gov/pubmed/26158096,55.686724,12.570072


In [24]:
# Muestra los datos

# Muestra todas las columnas y filas
dataDimension = literature.shape # obtiene las dimensiones de la tabla literature
pd.set_option("display.max_rows",5)
pd.set_option("display.max_columns",dataDimension[1])

# Muestra
literature

,algorithm_type,bibtex_id,author_1,author_2,author_3,author_4,author_5,author6,author_7,author_8,affiliation_last_author,city_last_author,country_last_author,title,journal,year,link_to_pubmed,latitude,longitude
0,active contours,Solloway_1997,﻿Solloway S,Hutchinson CE,Waterton JC,Taylor CJ,NaN,NaN,NaN,NaN,"Department of Medical Biophysics, University o...",Manchester,UK,﻿The use of active shape models for making thi...,﻿Magn Reson Med 37(6):943–952,1997,https://www.ncbi.nlm.nih.gov/pubmed/9178247,53.479489,-2.245115
1,active contours,Vincent_2011,﻿Vincent G,Wolstenholme C,Scott I,Bowes M,NaN,NaN,NaN,NaN,Imorphics Ltd.,Manchester,UK,﻿Fully Automatic Segmentation of the Knee Join...,﻿MICCAI 2010 Workshop Medical Image Analysis f...,2011,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,53.479489,-2.245115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,hybrid,Seim_2010,Seim H,Kainmueller D,Lamecker H,Bindernagel M,Malinowski J,Zachow S,NaN,NaN,"Zuse Institute Berlin (ZIB), Medical Planning ...",Berlin,Germany,﻿Model-based auto-segmentation of knee bones a...,﻿Medical Image Analysis for the Clinic: A Gran...,2010,NaN,52.517037,13.388860
28,hybrid,Dam_2015,Dam ﻿E B,Lillholm M,Marques J,Nielsen M,NaN,NaN,NaN,NaN,Biomediq and University of Copenhagen,Copenhagen,Denmark,Automatic segmentation of high- and low-field ...,"﻿SPIE Journal of Medical Imaging, vol. 2, pp....",2015,https://www.ncbi.nlm.nih.gov/pubmed/26158096,55.686724,12.570072


---
<a name = "manipulation"></a>
## 2. Automatizar la manipulación de datos

La manipulación de datos **automática** no compromete los datos originales y mantiene un registro de las manipulaciones, haciendo que los análisis sean reproducibles. Se *desaconseja* la manipulación **manual**, ya que compromete los datos originales, es propensa a errores y no mantiene un registro de los cambios, haciendo que los análisis sean difícilmente reproducibles.

- Cambiar el formato `bibtext_id` de  `author_year` a `author (year)` para una mejor legibilidad al pasar el cursor (por ejemplo, de `Solloway_1997` a `Solloway (1997)`).

In [25]:
#remplaza _ por espacio y abre parentesis
literature["bibtex_id"] = literature["bibtex_id"].str.replace('_',' (')
# agregar parentesis al final
literature["bibtex_id"] = literature["bibtex_id"].astype(str) + ")"  

# Muestra la tabla
literature

,algorithm_type,bibtex_id,author_1,author_2,author_3,author_4,author_5,author6,author_7,author_8,affiliation_last_author,city_last_author,country_last_author,title,journal,year,link_to_pubmed,latitude,longitude
0,active contours,Solloway (1997),﻿Solloway S,Hutchinson CE,Waterton JC,Taylor CJ,NaN,NaN,NaN,NaN,"Department of Medical Biophysics, University o...",Manchester,UK,﻿The use of active shape models for making thi...,﻿Magn Reson Med 37(6):943–952,1997,https://www.ncbi.nlm.nih.gov/pubmed/9178247,53.479489,-2.245115
1,active contours,Vincent (2011),﻿Vincent G,Wolstenholme C,Scott I,Bowes M,NaN,NaN,NaN,NaN,Imorphics Ltd.,Manchester,UK,﻿Fully Automatic Segmentation of the Knee Join...,﻿MICCAI 2010 Workshop Medical Image Analysis f...,2011,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4...,53.479489,-2.245115
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,hybrid,Seim (2010),Seim H,Kainmueller D,Lamecker H,Bindernagel M,Malinowski J,Zachow S,NaN,NaN,"Zuse Institute Berlin (ZIB), Medical Planning ...",Berlin,Germany,﻿Model-based auto-segmentation of knee bones a...,﻿Medical Image Analysis for the Clinic: A Gran...,2010,NaN,52.517037,13.388860
28,hybrid,Dam (2015),Dam ﻿E B,Lillholm M,Marques J,Nielsen M,NaN,NaN,NaN,NaN,Biomediq and University of Copenhagen,Copenhagen,Denmark,Automatic segmentation of high- and low-field ...,"﻿SPIE Journal of Medical Imaging, vol. 2, pp....",2015,https://www.ncbi.nlm.nih.gov/pubmed/26158096,55.686724,12.570072


---
<a name = "seeds"></a>
## 3. Definir semillas para generar números aleatorios

Los algoritmos crean números aleatorios partiendo de un número semilla (normalmente elegido del reloj del sistema) y siguiendo un conjunto de instrucciones que dan como resultado el número aleatorio deseado. **Fijar el número semilla** permite calcular siempre el mismo número aleatorio.

- Calcular la latitud y longitud de `city_last_author` y añadir un pequeño número aleatorio para evitar solapamientos en el mapa.

In [29]:
# Primera ejecución
np.random.seed(seed=3) # si esto no se pone, cada vez que se ejecute el código se obtendrán valores diferentes

# Adiciona un valor aleatorio a la latitud
random_lat = np.random.uniform(low=0.0, high=2.5, size=(len(literature["latitude"]),))
literature["latitude_random"] = literature["latitude"] + pd.Series(random_lat)

# adiciona un valor aleatorio a la longitud
random_lon = np.random.uniform(low=0.0, high=2.5, size=(len(literature["longitude"]),))
literature["longitude_random"] = literature["longitude"] + pd.Series(random_lon)

# show 
pd.set_option("display.max_rows",3) # muestra solo 3 filas
literature [["latitude", "latitude_random", "longitude", "longitude_random"]] # muestra solo las columnas latitude, latitude_random, longitude, longitude_random

,latitude,latitude_random,longitude,longitude_random
0,53.479489,54.856484,-2.245115,0.095844
...,...,...,...,...
28,55.686724,56.656652,12.570072,14.436938


Verificamos si en realidad estamos obteniendo los mismos números aleatorios cada vez que ejecutamos el código.

In [30]:
# Primera ejecución
np.random.seed(seed=3) # si esto no se pone, cada vez que se ejecute el código se obtendrán valores diferentes

# Adiciona un valor aleatorio a la latitud
random_lat = np.random.uniform(low=0.0, high=2.5, size=(len(literature["latitude"]),))
literature["latitude_random"] = literature["latitude"] + pd.Series(random_lat)

# adiciona un valor aleatorio a la longitud
random_lon = np.random.uniform(low=0.0, high=2.5, size=(len(literature["longitude"]),))
literature["longitude_random"] = literature["longitude"] + pd.Series(random_lon)

# show 
pd.set_option("display.max_rows",3) # muestra solo 3 filas
literature [["latitude", "latitude_random", "longitude", "longitude_random"]] # muestra solo las columnas latitude, latitude_random, longitude, longitude_random

,latitude,latitude_random,longitude,longitude_random
0,53.479489,54.856484,-2.245115,0.095844
...,...,...,...,...
28,55.686724,56.656652,12.570072,14.436938


## Visualize map
- Show the interactive map

In [31]:
#importa las coordenadas de los países para crear el mapa de fondo
countries = alt.topo_feature(data.world_110m.url, 'countries')

# crea el mapa de fondo
background = alt.Chart(countries).mark_geoshape(
    fill        = 'white',
    stroke      = 'lightgray',
    strokeWidth = 1.5
).project(
    "equirectangular"
).properties(
    width  = 1250,
    height = 750
)

# crea los puntos
points = alt.Chart(literature).mark_circle().encode(
    longitude = 'longitude_random:Q',
    latitude  = 'latitude_random:Q',
    size      = alt.value(100),
    color     = 'algorithm_type',
    tooltip   = 'bibtex_id' # name of each point when hovering
    
)

# Visualiza el mapa
background + points 

alt.LayerChart(...)

<a name="dependencies"></a>
## 4. Veamos las dependencias

Las dependencias son fundamentales para guardar el **entorno computacional**.
 
- Usamos [watermark](https://github.com/rasbt/watermark) para imprimir la versión de python, ipython y paquetes, y las características de la computadora

In [ ]:
%load_ext watermark

# python, ipython, librerias, y sistema operativo
%watermark -v -m -p wget,pandas,numpy,geopy,altair,vega,vega_datasets,watermark 

# date
print (" ")
%watermark -u -n -t -z 

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Python implementation: CPython
Python version       : 3.10.14
IPython version      : 8.22.2

wget         : 3.2
pandas       : 1.5.3
numpy        : 1.26.4
geopy        : not installed
altair       : 5.3.0
vega         : 4.0.0
vega_datasets: 0.9.0
watermark    : 2.4.3

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
CPU cores   : 16
Architecture: 64bit

 
Last updated: Thu Jul 04 2024 18:18:04Romance Daylight Time



## Otra forma de exportar los requisitos del entorno computacional

In [35]:
import os
os.system('pip freeze > requirements.txt')

0